## Mixed Model Equations(MME) from Data and Model

### Extensions for Models with SNP Covariates

We have already seen how the MME can be constructed from the Data in a DataFrame and a higher-level description of the model. The model description included a string representation of the model equation, identification of the quantitative factors in the model, and specification of the random terms in the model. Here, we will see how substitution effects of SNPs can be inluded in the model as random effects in the model. In multiple-trait analyses, a single command will be used to include SNP effects in all the models. 

### Structures for Building MME

In [1]:
#EXECUTE 
using DataFrames, SparseArrays, LinearAlgebra, Random, JWAS, CSV, Distributions
using Printf

mutable struct ModelTerm
    trmString::AbstractString
    iModel::Int64                           # the model (trait) this term belongs to
    startPos::Int64
    endPos::Int64
    randomType::String
end

# Composite type for including SNP effects in the model
mutable struct Genotypes                    # ZM where Z is a sparse incidence matrix and M is a dense matrix of covariates           
    trmString::AbstractString               # "Ztrm"
    Z::SparseMatrixCSC{Float64,Int64}       # incidence matrix Z to map rows of M to phenotypes according to "Ind" 
    M::Array{Float64,2}                     # dense matrix of covariates
    MArray::Array{Any,1}                    # column views of M
    MPMArray::Array{Float64,1}              # diagonals of M'M
    idGeno::Array{String}                   # row names of M (Ind must be a subset of idGeno)
    α::Array{Float64,2}                     # nCovariates x nTraits matrix of effects
    αMean::Array{Float64,2}                 # mean of sampled values  
    yAdjArray::Array{Any,1}                 # Array of Arrays that will be used to access adjusted y-vector
    νpr::Float64
    Spr::Array{Float64,2}
    Vi::Array{Float64,2}
    meanV::Array{Float64,2}
    estimate::Bool
    outSamples::Bool
    outFreq::Int64
    outStream   
end 

mutable struct RandomEffect
    modelTermVec::Array{ModelTerm,1}      
    νpr::Float64
    Spr::Array{Float64,2}
    Vi::Array{Float64,2}
    meanV::Array{Float64,2}
    Ai
    estimate::Bool
    outSamples::Bool
    outFreq::Int64
    outStream
end

mutable struct MME
    modelEquations::Array{AbstractString,1} #"[y1 = A + B + A*B ;y2 = A + B"]
    covVec::Array{String}                   #["age"]
    modelTermVec::Array{ModelTerm,1}        #[modelTerm("A") , modelTerm("A*B")]
    modelTermDict::Dict{AbstractString,ModelTerm}
    genotypes                               # dense component of model
    randomEffectsVec::Array{RandomEffect,1} #[RandomEffect("Ind"), RandomEffect("Mat")]
    ped                                     # false or a pedigee structure defined in JWAS.PedModule
    depVarVec::Array{AbstractString,1}      #["y1","y2"]
    X::SparseMatrixCSC{Float64,Int64}       #design matrix
    mmeLhs::SparseMatrixCSC{Float64,Int64}  #left-hand side for MME
    mmeRhs::Array{Float64,1}                #right-hand side for MME
    mmeSpl::Array{Float64,1}                #sample of location parameters
    meanEffects::Array{Float64,1}           #mean of location parameters
    varRes::Array{Float64,2}                #residual variance 
    νRes::Float64                           #prior degrees of freedom for varRes
    SRes::Array{Float64,2}                  #prior scale parameter for resVar
    y::Array{Float64,1}                     #dependent variable
    yAdj::Array{Float64,1}
    meanVarRes::Array{Float64,2}                     
end 

function ModelTerm(str::AbstractString,iModel)
    ModelTerm(str,iModel,0,0,"fixed")
end  

ModelTerm

### Functions for Building MME

In [13]:
#EXECUTE 
function initMME(modelEquations::AbstractString,varRes;νRes=5.0)  # "y1 = A + A*B; y2 = A + B + !(A)M
    if modelEquations==""
        error("modelEquations is empty\n")
    end
    modelVec   = [strip(i) for i in split(modelEquations,[';','\n'],keepempty=false)]
    nModels    = size(modelVec,1)
    depVarVec  = Array{AbstractString,1}()
    modelTermVec  = Array{ModelTerm,1}()                 # modelTerms across all the models
    modelTermDict = Dict{AbstractString,ModelTerm}()  
    for (m,model) = enumerate(modelVec)
        leftRight  = split(model,"=")                 # "y", "A+A*B"
        depVarVec  = [depVarVec; strip(leftRight[1])] # ["y1","y2"]
        modelParts = strip(leftRight[2])              # "A+A*B" 
        termsVec   = split(modelParts,"+")            # "A","A*B"
        modelTermVeci = [ModelTerm(string(m)*":"*strip(trmStr),m) for trmStr in termsVec]
        modelTermVec  = [modelTermVec;modelTermVeci]    
    end
    for i in modelTermVec
        modelTermDict[i.trmString] = i
    end
    genotypes = false
    randomEffectsVec = []
    ped = false
    covVec = []
    X = spzeros(0,0)
    mmeLhs = spzeros(0,0)
    mmeRhs = []
    mmeSpl = []
    meanEffects = []
    y      = Array{Float64,1}(undef,0)
    yAdj   = Array{Float64,1}(undef,0)
    m = size(varRes,1)
    varRes = m==1 ? fill(varRes,1,1) : varRes
    SRes   = varRes*(νRes - m - 1)
    meanVarRes = zero(varRes)
    return MME(modelVec,covVec,modelTermVec,modelTermDict,genotypes,randomEffectsVec,ped,
               depVarVec,X,mmeLhs,mmeRhs,mmeSpl,meanEffects,varRes,νRes,SRes,y,yAdj,meanVarRes)
end 

initMME (generic function with 1 method)

### Function for Adding SNP Effects to Model

In [2]:
#EXECUTE
function addGenotypes!(mme::MME,df::DataFrame,ZTrm::AbstractString,M::Array{Float64,2},
        idGeno::Array{String,1},V;
        estimate::Bool=false,
        νPrior::Float64=4.1,
        outSamples::Bool=false,
        outFreq::Int64=100)
        m = size(V,1)        
        if outSamples==true
            fileName = "G.samples"
            outStream = open(fileName,"w")
            for i=1:m,j=i:m
                if i==j==1
                    @printf(outStream,"%6s", "V[$i,$j]")
                else
                    @printf(outStream," %6s", "V[$i,$j]")    
                end        
            end
            @printf(outStream,"\n")        
        else
            outStream = nothing
        end        
        var = m==1 ? fill(V,1,1) : V
        d = Dict()
        for (i,s) = enumerate(idGeno)
            d[s] = i
        end
        n = size(df,1) 
        ii = collect(1:n) 
        jj = Array{Int64,1}(undef,n)
        idPheno = string.(df[!,Symbol(ZTrm)])    
        for (i,v) = enumerate(idPheno)
            jj[i] = d[v]
        end
        Z = sparse(ii,jj,1.0,n,size(idGeno,1))
        α = zeros(0,0)
        αMean = zeros(0,0)    
        Spr = (νPrior - m - 1)*var 
        yAdjArray = Array{Array{Float64,1},1}(undef,0)
        MArray = Array{Any,1}(undef,0)
        MPMArray = Array{Any,1}(undef,0)  
        mme.genotypes = Genotypes(ZTrm,Z,M,MArray,MPMArray,idGeno,α,αMean,yAdjArray,νPrior,Spr,
            inv(var),zero(var),estimate,outSamples,outFreq,outStream)
        return
end

addGenotypes! (generic function with 1 method)

In [15]:
#EXECUTE 
# This function returns returns a dictionary with the names in the pedigree as the keys and their 
# sequential numbers as the associated values
# It also returns vector with the keys in sequential order. 

function mkDict(ped::JWAS.PedModule.Pedigree)
    d = Dict()
    names = Array{String}(undef,length(ped.idMap))
    for i in ped.idMap    
        d[i.first] = i.second.seqID
        names[i.second.seqID] = i.first
    end
    return d,names
end

# This function returns 
# a dictionary with the unique values in the vector "a" as the keys and their 
# sequential numbers as the associated values
# It also returns vector with the keys in sequential order. 
function mkDict(a)
    d = Dict()
    aUnique = unique(a)
    names = Array{String}(undef,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
    names[i] = s
    d[s] = i
    end
    return d,names
end

function getX(mme,modelTerm::ModelTerm,covariables,df)
    n = size(df,1)
    nModels = size(mme.modelEquations,1)
    trmString = split(modelTerm.trmString,':')[2]
    if trmString == "intercept"
        ii = (modelTerm.iModel-1)*n .+ (1:n) # row numbers
        jj = ones(n)
        val = 1.0
        p = 1
        X  = sparse(ii,jj,val,n*nModels,p)
        colNames = modelTerm.trmString
        return X,colNames
    end
    factors = strip.(split(trmString,"*"))
    covs = [i in covariables for i in factors]
    
    if covs[1] == false
        str = string.(df[:,Symbol(factors[1])])
        val = 1.0
    else
        str = fill(factors[1],n) 
        val = df[:,Symbol(factors[1])]    
    end       

    for i in 2:length(factors)
        if covs[i] == false
            str = str .*" x ".*string.(df[:,Symbol(factors[i])])
            val = val .* 1.0 
        else
            str = str .*" x ".*fill(factors[i],n) 
            val = val .* df[:,Symbol(factors[i])]    
        end 
    end
    dict,colNames   = modelTerm.randomType != "A" ? mkDict(str) : mkDict(mme.ped)
    str = modelTerm.randomType != "A" ? str : string.(df[:,Symbol(factors[1])])        
    ii = (modelTerm.iModel-1)*n .+ (1:n)      # row numbers 
    jj = [dict[i] for i in str]  # column numbers
    p = length(colNames)        
    X  = sparse(ii,jj,val,n*nModels,p)
    return X, strip(modelTerm.trmString)*": ".*colNames   
end

function setRandom!(mme::MME,trmStrings::String,V,ped=false;
                estimate::Bool=false,
                νPrior::Float64=4.1,
                outSamples::Bool=false,
                outFreq::Int64=100
            )
    m = size(V,1)        
    if outSamples==true
        fileName = "V($trmStrings).samples"
        outStream = open(fileName,"w")
        for i=1:m,j=i:m
            if i==j==1
                @printf(outStream,"%6s", "V[$i,$j]")
            else
                @printf(outStream," %6s", "V[$i,$j]")    
            end        
        end
        @printf(outStream,"\n")        
    else
        outStream = nothing
    end        
    var = m==1 ? fill(V,1,1) : V        
    trmVec = strip.(split(trmStrings,[',';' '],keepempty=false))
    modelTerms = []
    for modelTerm in mme.modelTermVec
        trmString = split(modelTerm.trmString,':')[2]
        if trmString in trmVec
            push!(modelTerms,modelTerm)
        end                
    end
    if length(modelTerms) != size(var,1)
        println("size of var: $(size(var,1)) does not match the number of random effects: $(length(modelTerms))")
        return
    end 
                
    if ped==false
        [trm.randomType = "I" for trm in modelTerms]
        Ai = I
    else
        [trm.randomType = "A" for trm in modelTerms] 
        mme.ped = pedigree
        Ai = JWAS.PedModule.AInverse(pedigree)    
    end
    Spr = (νPrior - m - 1)*var    
    randomEffect = RandomEffect(modelTerms,νPrior,Spr,inv(var),zero(var),Ai,
                        estimate,outSamples,outFreq,outStream)
    push!(mme.randomEffectsVec,randomEffect)   
end

function addGiMats!(mme::MME)
    for randomEffect in mme.randomEffectsVec
        for (i,modelTermi) in enumerate(randomEffect.modelTermVec), (j,modelTermj) in enumerate(randomEffect.modelTermVec)
            starti = modelTermi.startPos
            startj = modelTermj.startPos
            endi = modelTermi.endPos
            endj = modelTermj.endPos
            mme.mmeLhs[starti:endi,startj:endj] = mme.mmeLhs[starti:endi,startj:endj] + randomEffect.Ai*randomEffect.Vi[i,j]
        end
    end
end

addGiMats! (generic function with 1 method)

In [58]:
#EXECUTE
function getLhsRhs!(mme,df)
    n = size(df,1)
    X,colNames = getX(mme,mme.modelTermVec[1],mme.covVec,df)
    mme.modelTermVec[1].startPos = 1
    mme.modelTermVec[1].endPos  = mme.modelTermVec[1].startPos + size(X,2) - 1
    for i = 2:size(mme.modelTermVec,1)
        Xi,namesi = getX(mme,mme.modelTermVec[i],mme.covVec,df)
        X = [X Xi]
        mme.modelTermVec[i].startPos = mme.modelTermVec[i-1].endPos + 1
        mme.modelTermVec[i].endPos   = mme.modelTermVec[i].startPos + size(Xi,2) - 1
        colNames = [colNames; namesi]
    end
    y = Array{Float64,1}()
    for v in mme.depVarVec
        y = [y;df[:,Symbol(v)]]
    end
    Ri = kron(inv(mme.varRes),sparse(I, n, n))
    mme.X = X
    mme.mmeLhs = X'Ri*X
    mme.mmeRhs = X'Ri*y
    mme.mmeSpl = zero(mme.mmeRhs)
    mme.meanEffects = zero(mme.mmeRhs)
    mme.y      = y
    mme.yAdj = copy(mme.y)
    addGiMats!(mme)
    if mme.genotypes != false
        mme.genotypes.M = mme.genotypes.M .- mean(mme.genotypes.M,dims=1)
        mme.genotypes.M = mme.genotypes.Z * mme.genotypes.M
        nModels  = size(mme.varRes,1)
        nMarkers = size(mme.genotypes.M,2)
        yAdjArray = Array{Any,1}(undef,nModels)
        for i=1:nModels
            startPos = (i-1)*n + 1
            endPos = startPos + n - 1
            yAdjArray[i] = @view mme.yAdj[startPos:endPos]
        end
        MArray   = Array{Any,1}(undef,nMarkers)
        MPMArray = Array{Float64,1}(undef,nMarkers)
        for i=1:nMarkers
            m = @view mme.genotypes.M[:,i]
            MArray[i] = m
            MPMArray[i] = m'm
        end
        mme.genotypes.yAdjArray = yAdjArray
        mme.genotypes.MArray = MArray
        mme.genotypes.MPMArray = MPMArray
        mme.genotypes.α = zeros(nMarkers,nModels)
        mme.genotypes.αMean = zeros(nMarkers,nModels)
    end
    return mme.mmeLhs,mme.mmeRhs,colNames
end
function updateLhsRhs!(mme)
    m = size(mme.varRes,1)
    n = Int(size(mme.y,1)/m)
    Ri = kron(inv(mme.varRes),sparse(I, n, n))
    mme.mmeLhs = mme.X'Ri*mme.X
    mme.mmeRhs = mme.X'Ri*mme.yAdj
    addGiMats!(mme)
end

updateLhs! (generic function with 1 method)

In [9]:
pedigree   = get_pedigree("pedFile",separator=",",header=false);

The delimiter in pedFile is ','.


coding pedigree... 100%|████████████████████████████████| Time: 0:00:00
calculating inbreeding... 100%|█████████████████████████| Time: 0:00:00


Finished!


In [10]:
data = CSV.read("data.phen")

,Ind,Mat,y1,y2,x
,Int64,Int64,Float64,Float64,Float64
1,3,2,8.9,9.2,11.9
2,4,2,9.7,5.7,10.8
3,5,4,8.8,8.5,11.7


In [59]:
varRes = 1.0
mme = initMME("y1 = intercept + Ind",varRes);
setRandom!(mme,"Ind",1.0,pedigree);

In [60]:
M = float.(rand(Binomial(2,0.5),5,10))

5×10 Array{Float64,2}:
 1.0  2.0  1.0  0.0  2.0  1.0  2.0  1.0  1.0  1.0
 1.0  2.0  2.0  1.0  2.0  0.0  0.0  0.0  0.0  1.0
 2.0  2.0  1.0  1.0  1.0  2.0  1.0  2.0  2.0  1.0
 2.0  0.0  2.0  1.0  1.0  2.0  1.0  0.0  2.0  1.0
 1.0  0.0  1.0  2.0  0.0  1.0  1.0  0.0  2.0  2.0

In [61]:
idGeno = string.(1:5);
addGenotypes!(mme,data,"Ind",M,idGeno,1.0)

In [62]:
lhs,rhs,names = getLhsRhs!(mme,data)
[names Matrix(lhs) rhs]

6×8 Array{Any,2}:
 "1:intercept"  3.0   0.0   0.0   1.0   1.0   1.0  27.4
 "1:Ind: 1"     0.0   2.0   1.0  -1.0  -1.0   0.0   0.0
 "1:Ind: 2"     0.0   1.0   2.0  -1.0  -1.0   0.0   0.0
 "1:Ind: 4"     1.0  -1.0  -1.0   3.5   0.5  -1.0   9.7
 "1:Ind: 3"     1.0  -1.0  -1.0   0.5   3.5  -1.0   8.9
 "1:Ind: 5"     1.0   0.0   0.0  -1.0  -1.0   3.0   8.8

In [63]:
mme.genotypes.M

3×10 Array{Float64,2}:
  0.6   0.8  -0.4  0.0  -0.2   0.8  0.0   1.4  0.6  -0.2
  0.6  -1.2   0.6  0.0  -0.2   0.8  0.0  -0.6  0.6  -0.2
 -0.4  -1.2  -0.4  1.0  -1.2  -0.2  0.0  -0.6  0.6   0.8

In [64]:
mean(mme.genotypes.M,dims=1)

1×10 Array{Float64,2}:
 0.266667  -0.533333  -0.0666667  0.333333  …  0.0  0.0666667  0.6  0.133333

In [65]:
mme.genotypes.MArray

10-element Array{Any,1}:
 [0.6, 0.6, -0.4]  
 [0.8, -1.2, -1.2] 
 [-0.4, 0.6, -0.4] 
 [0.0, 0.0, 1.0]   
 [-0.2, -0.2, -1.2]
 [0.8, 0.8, -0.2]  
 [0.0, 0.0, 0.0]   
 [1.4, -0.6, -0.6] 
 [0.6, 0.6, 0.6]   
 [-0.2, -0.2, 0.8] 

In [66]:
mme.genotypes.MPMArray

10-element Array{Float64,1}:
 0.8799999999999998
 3.5200000000000005
 0.68              
 1.0               
 1.5200000000000005
 1.3199999999999994
 0.0               
 2.6799999999999997
 1.0799999999999994
 0.7199999999999999

In [22]:
Matrix(mme.genotypes.Z)

3×5 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [57]:
mme.genotypes.yAdjArray[1]'

1×3 Adjoint{Float64,SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true}}:
 8.9  9.7  8.8

In [22]:
Matrix(mme.randomEffectsVec[1].Ai)

5×5 Array{Float64,2}:
  2.0   1.0  -1.0  -1.0   0.0
  1.0   2.0  -1.0  -1.0   0.0
 -1.0  -1.0   2.5   0.5  -1.0
 -1.0  -1.0   0.5   2.5  -1.0
  0.0   0.0  -1.0  -1.0   2.0

In [46]:
mme.genotypes.yAdjArray

1-element Array{Array{Float64,1},1}:
 [8.9, 9.7, 8.8]

In [18]:
Matrix(mme.X'mme.X)

6×6 Array{Float64,2}:
 3.0  0.0  0.0  1.0  1.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0

In [19]:
mme.modelTermVec

2-element Array{ModelTerm,1}:
 ModelTerm("intercept", 1, 1, "fixed")
 ModelTerm("Ind", 2, 6, "A")          

In [20]:
varRes = 1.0
mme = initMME("y1 = intercept + Ind + Mat",varRes);
setRandom!(mme,"Ind",1.0,pedigree)
setRandom!(mme,"Mat",1.0,pedigree);

In [21]:
lhs,rhs,names = getLhsRhs!(mme,data)
[names Matrix(lhs)]

11×12 Array{Any,2}:
 "intercept"  3.0   0.0   0.0   1.0   1.0   1.0   0.0   2.0   1.0   0.0   0.0
 "Ind: 1"     0.0   2.0   1.0  -1.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0
 "Ind: 2"     0.0   1.0   2.0  -1.0  -1.0   0.0   0.0   0.0   0.0   0.0   0.0
 "Ind: 4"     1.0  -1.0  -1.0   3.5   0.5  -1.0   0.0   1.0   0.0   0.0   0.0
 "Ind: 3"     1.0  -1.0  -1.0   0.5   3.5  -1.0   0.0   1.0   0.0   0.0   0.0
 "Ind: 5"     1.0   0.0   0.0  -1.0  -1.0   3.0   0.0   0.0   1.0   0.0   0.0
 "Mat: 1"     0.0   0.0   0.0   0.0   0.0   0.0   2.0   1.0  -1.0  -1.0   0.0
 "Mat: 2"     2.0   0.0   0.0   1.0   1.0   0.0   1.0   4.0  -1.0  -1.0   0.0
 "Mat: 4"     1.0   0.0   0.0   0.0   0.0   1.0  -1.0  -1.0   3.5   0.5  -1.0
 "Mat: 3"     0.0   0.0   0.0   0.0   0.0   0.0  -1.0  -1.0   0.5   2.5  -1.0
 "Mat: 5"     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0  -1.0   2.0

In [22]:
Matrix(mme.X'mme.X)

11×11 Array{Float64,2}:
 3.0  0.0  0.0  1.0  1.0  1.0  0.0  2.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  0.0  0.0  1.0  1.0  0.0  0.0  2.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [37]:
V = [1   0.5
     0.5 2.0]

2×2 Array{Float64,2}:
 1.0  0.5
 0.5  2.0

In [35]:
varRes = 1.0
mme = initMME("y1 = intercept + Ind + Mat",varRes);
setRandom!(mme,"Ind, Mat",1.0,pedigree);

size of var: 1 does not match the number of random effects: 2


In [38]:
varRes = 1.0
mme = initMME("y1 = intercept + Ind + Mat",varRes);
setRandom!(mme,"Ind, Mat",V,pedigree);

In [39]:
lhs,rhs,names = getLhsRhs!(mme,data)
Matrix(lhs)

11×11 Array{Float64,2}:
 3.0   0.0        0.0        1.0       …   1.0        0.0        0.0     
 0.0   2.28571    1.14286   -1.14286       0.285714   0.285714   0.0     
 0.0   1.14286    2.28571   -1.14286       0.285714   0.285714   0.0     
 1.0  -1.14286   -1.14286    3.85714      -0.714286  -0.142857   0.285714
 1.0  -1.14286   -1.14286    0.571429     -0.142857  -0.714286   0.285714
 1.0   0.0        0.0       -1.14286   …   1.28571    0.285714  -0.571429
 0.0  -0.571429  -0.285714   0.285714     -0.571429  -0.571429   0.0     
 2.0  -0.285714  -0.571429   1.28571      -0.571429  -0.571429   0.0     
 1.0   0.285714   0.285714  -0.714286      2.42857    0.285714  -0.571429
 0.0   0.285714   0.285714  -0.142857      0.285714   1.42857   -0.571429
 0.0   0.0        0.0        0.285714  …  -0.571429  -0.571429   1.14286 

In [41]:
varRes = 1.0
mme = initMME("y1 = intercept + Ind + Ind*x",varRes);
mme.covVec = ["x"]
setRandom!(mme,"Ind",1.0,pedigree);

In [42]:
size(mme.randomEffectsVec,1)

1

In [43]:
setRandom!(mme,"Ind*x",1.0,pedigree);

In [44]:
size(mme.randomEffectsVec,1)

2

In [45]:
lhs,rhs,names = getLhsRhs!(mme,data)
[names Matrix(lhs)]

11×12 Array{Any,2}:
 "1:intercept"   3.0   0.0   0.0   1.0  …   0.0   0.0   10.8    11.9    11.7 
 "1:Ind: 1"      0.0   2.0   1.0  -1.0      0.0   0.0    0.0     0.0     0.0 
 "1:Ind: 2"      0.0   1.0   2.0  -1.0      0.0   0.0    0.0     0.0     0.0 
 "1:Ind: 4"      1.0  -1.0  -1.0   3.5      0.0   0.0   10.8     0.0     0.0 
 "1:Ind: 3"      1.0  -1.0  -1.0   0.5      0.0   0.0    0.0    11.9     0.0 
 "1:Ind: 5"      1.0   0.0   0.0  -1.0  …   0.0   0.0    0.0     0.0    11.7 
 "1:Ind*x: 1"    0.0   0.0   0.0   0.0      2.0   1.0   -1.0    -1.0     0.0 
 "1:Ind*x: 2"    0.0   0.0   0.0   0.0      1.0   2.0   -1.0    -1.0     0.0 
 "1:Ind*x: 4"   10.8   0.0   0.0  10.8     -1.0  -1.0  119.14    0.5    -1.0 
 "1:Ind*x: 3"   11.9   0.0   0.0   0.0     -1.0  -1.0    0.5   144.11   -1.0 
 "1:Ind*x: 5"   11.7   0.0   0.0   0.0  …   0.0   0.0   -1.0    -1.0   138.89

In [32]:
Matrix(mme.X'mme.X)

11×11 Array{Float64,2}:
  3.0  0.0  0.0   1.0   1.0   1.0  0.0  0.0   10.8    11.9    11.7 
  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0    0.0     0.0     0.0 
  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0    0.0     0.0     0.0 
  1.0  0.0  0.0   1.0   0.0   0.0  0.0  0.0   10.8     0.0     0.0 
  1.0  0.0  0.0   0.0   1.0   0.0  0.0  0.0    0.0    11.9     0.0 
  1.0  0.0  0.0   0.0   0.0   1.0  0.0  0.0    0.0     0.0    11.7 
  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0    0.0     0.0     0.0 
  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0    0.0     0.0     0.0 
 10.8  0.0  0.0  10.8   0.0   0.0  0.0  0.0  116.64    0.0     0.0 
 11.9  0.0  0.0   0.0  11.9   0.0  0.0  0.0    0.0   141.61    0.0 
 11.7  0.0  0.0   0.0   0.0  11.7  0.0  0.0    0.0     0.0   136.89

In [46]:
varRes = 1.0
mme = initMME("y1 = intercept + Ind + Ind*x",varRes);
mme.covVec = ["x"]
setRandom!(mme,"Ind, Ind*x",V,pedigree);

In [47]:
size(mme.randomEffectsVec,1)

1

In [48]:
lhs,rhs,names = getLhsRhs!(mme,data)
[names Matrix(lhs) rhs]

11×13 Array{Any,2}:
 "1:intercept"   3.0   0.0        0.0       …   11.9        11.7        27.4 
 "1:Ind: 1"      0.0   2.28571    1.14286        0.285714    0.0         0.0 
 "1:Ind: 2"      0.0   1.14286    2.28571        0.285714    0.0         0.0 
 "1:Ind: 4"      1.0  -1.14286   -1.14286       -0.142857    0.285714    9.7 
 "1:Ind: 3"      1.0  -1.14286   -1.14286       11.1857      0.285714    8.9 
 "1:Ind: 5"      1.0   0.0        0.0       …    0.285714   11.1286      8.8 
 "1:Ind*x: 1"    0.0  -0.571429  -0.285714      -0.571429    0.0         0.0 
 "1:Ind*x: 2"    0.0  -0.285714  -0.571429      -0.571429    0.0         0.0 
 "1:Ind*x: 4"   10.8   0.285714   0.285714       0.285714   -0.571429  104.76
 "1:Ind*x: 3"   11.9   0.285714   0.285714     143.039      -0.571429  105.91
 "1:Ind*x: 5"   11.7   0.0        0.0       …   -0.571429  138.033     102.96

In [49]:
Matrix(mme.X)

3×11 Array{Float64,2}:
 1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   0.0  11.9   0.0
 1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  10.8   0.0   0.0
 1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   0.0   0.0  11.7

In [50]:
Matrix(mme.X'mme.X)

11×11 Array{Float64,2}:
  3.0  0.0  0.0   1.0   1.0   1.0  0.0  0.0   10.8    11.9    11.7 
  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0    0.0     0.0     0.0 
  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0    0.0     0.0     0.0 
  1.0  0.0  0.0   1.0   0.0   0.0  0.0  0.0   10.8     0.0     0.0 
  1.0  0.0  0.0   0.0   1.0   0.0  0.0  0.0    0.0    11.9     0.0 
  1.0  0.0  0.0   0.0   0.0   1.0  0.0  0.0    0.0     0.0    11.7 
  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0    0.0     0.0     0.0 
  0.0  0.0  0.0   0.0   0.0   0.0  0.0  0.0    0.0     0.0     0.0 
 10.8  0.0  0.0  10.8   0.0   0.0  0.0  0.0  116.64    0.0     0.0 
 11.9  0.0  0.0   0.0  11.9   0.0  0.0  0.0    0.0   141.61    0.0 
 11.7  0.0  0.0   0.0   0.0  11.7  0.0  0.0    0.0     0.0   136.89

In [51]:
varRes = 1.0
mme = initMME("y1 = intercept + Ind + Ind*x",varRes);
mme.covVec = ["x"]
setRandom!(mme,"Ind",1.0,pedigree)
setRandom!(mme,"Ind*x",1.0);

In [52]:
lhs,rhs,names = getLhsRhs!(mme,data)
[names Matrix(lhs) rhs]

9×11 Array{Any,2}:
 "1:intercept"      3.0   0.0   0.0   1.0  …   11.9    10.8    11.7    27.4 
 "1:Ind: 1"         0.0   2.0   1.0  -1.0       0.0     0.0     0.0     0.0 
 "1:Ind: 2"         0.0   1.0   2.0  -1.0       0.0     0.0     0.0     0.0 
 "1:Ind: 4"         1.0  -1.0  -1.0   3.5       0.0    10.8     0.0     9.7 
 "1:Ind: 3"         1.0  -1.0  -1.0   0.5      11.9     0.0     0.0     8.9 
 "1:Ind: 5"         1.0   0.0   0.0  -1.0  …    0.0     0.0    11.7     8.8 
 "1:Ind*x: 3 x x"  11.9   0.0   0.0   0.0     142.61    0.0     0.0   105.91
 "1:Ind*x: 4 x x"  10.8   0.0   0.0  10.8       0.0   117.64    0.0   104.76
 "1:Ind*x: 5 x x"  11.7   0.0   0.0   0.0       0.0     0.0   137.89  102.96

In [74]:
varRes =[1   0.5
         0.5 2.0]
V = [1.0 0.0 0.0
     0.0 2.0 0.0
     0.0 0.0 3.0]
mme = initMME("y1 = intercept + Ind + Ind*x;
               y2 = intercept + Ind",varRes);
mme.covVec = ["x"]
#setRandom!(mme,"Ind Ind*x",V,pedigree);

In [75]:
lhs,rhs,names = getLhsRhs!(mme,data)
[names Matrix(lhs)]

11×12 Array{Any,2}:
 "1:intercept"      3.42857    1.14286   …  -0.285714  -0.285714  -0.285714
 "1:Ind: 3"         1.14286    1.14286      -0.285714   0.0        0.0     
 "1:Ind: 4"         1.14286    0.0           0.0       -0.285714   0.0     
 "1:Ind: 5"         1.14286    0.0           0.0        0.0       -0.285714
 "1:Ind*x: 3 x x"  13.6       13.6          -3.4        0.0        0.0     
 "1:Ind*x: 4 x x"  12.3429     0.0       …   0.0       -3.08571    0.0     
 "1:Ind*x: 5 x x"  13.3714     0.0           0.0        0.0       -3.34286 
 "2:intercept"     -0.857143  -0.285714      0.571429   0.571429   0.571429
 "2:Ind: 3"        -0.285714  -0.285714      0.571429   0.0        0.0     
 "2:Ind: 4"        -0.285714   0.0           0.0        0.571429   0.0     
 "2:Ind: 5"        -0.285714   0.0       …   0.0        0.0        0.571429

In [72]:
mme.randomEffectsVec[1].modelTermVec

3-element Array{ModelTerm,1}:
 ModelTerm("1:Ind", 1, 2, 6, "A")   
 ModelTerm("1:Ind*x", 1, 7, 11, "A")
 ModelTerm("2:Ind", 2, 13, 17, "A") 

In [73]:
inv(V)

3×3 Array{Float64,2}:
 1.0  0.0  0.0     
 0.0  0.5  0.0     
 0.0  0.0  0.333333

In [18]:
varRes =[1   0.5
         0.5 2.0]
V = [1.0 0.0 0.0
     0.0 2.0 0.0
     0.0 0.0 3.0]
mme = initMME("y1 = intercept + Ind + Ind*x
               y2 = intercept + Ind",varRes);
mme.covVec = ["x"]
setRandom!(mme,"Ind Ind*x",V,pedigree);

In [19]:
mme.randomEffectsVec[1].modelTermVec

3-element Array{ModelTerm,1}:
 ModelTerm("1:Ind", 1, 0, 0, "A")  
 ModelTerm("1:Ind*x", 1, 0, 0, "A")
 ModelTerm("2:Ind", 2, 0, 0, "A")  

In [10]:
lhs,rhs,names = getLhsRhs!(mme,data)
[names Matrix(lhs)]

17×18 Array{Any,2}:
 "1:intercept"   3.42857    0.0   0.0  …  -0.285714  -0.285714  -0.285714
 "1:Ind: 1"      0.0        2.0   1.0      0.0        0.0        0.0     
 "1:Ind: 2"      0.0        1.0   2.0      0.0        0.0        0.0     
 "1:Ind: 4"      1.14286   -1.0  -1.0     -0.285714   0.0        0.0     
 "1:Ind: 3"      1.14286   -1.0  -1.0      0.0       -0.285714   0.0     
 "1:Ind: 5"      1.14286    0.0   0.0  …   0.0        0.0       -0.285714
 "1:Ind*x: 1"    0.0        0.0   0.0      0.0        0.0        0.0     
 "1:Ind*x: 2"    0.0        0.0   0.0      0.0        0.0        0.0     
 "1:Ind*x: 4"   12.3429     0.0   0.0     -3.08571    0.0        0.0     
 "1:Ind*x: 3"   13.6        0.0   0.0      0.0       -3.4        0.0     
 "1:Ind*x: 5"   13.3714     0.0   0.0  …   0.0        0.0       -3.34286 
 "2:intercept"  -0.857143   0.0   0.0      0.571429   0.571429   0.571429
 "2:Ind: 1"      0.0        0.0   0.0     -0.333333  -0.333333   0.0     
 "2:Ind: 2"      0

In [16]:
varRes =[1   0.5
         0.5 2.0]
V = [1.0 0.0
     0.0 2.0 ]
mme = initMME("y1 = intercept + Ind + Ind*x
               y2 = intercept + Ind",varRes);
mme.covVec = ["x"]
setRandom!(mme,"Ind",V,pedigree);

In [17]:
mme.randomEffectsVec[1].modelTermVec

2-element Array{ModelTerm,1}:
 ModelTerm("1:Ind", 1, 0, 0, "A")
 ModelTerm("2:Ind", 2, 0, 0, "A")

In [2]:
y = float.(1:10)

10-element Array{Float64,1}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.0
  8.0
  9.0
 10.0

In [30]:
yArray = []
push!(yArray,unsafe_wrap(Array,pointer(y,1),5))
push!(yArray,unsafe_wrap(Array,pointer(y,6),5))

2-element Array{Any,1}:
 [1.0, 2.0, 3.0, 4.0, 5.0] 
 [6.0, 7.0, 8.0, 9.0, 10.0]

In [32]:
yArray[2][2] = 7.7
y

10-element Array{Float64,1}:
  1.0
  2.0
  3.0
  4.0
  5.0
  6.0
  7.7
  8.0
  9.0
 10.0

In [3]:
v = []
push!(v,@view y[1:5])

1-element Array{Any,1}:
 [1.0, 2.0, 3.0, 4.0, 5.0]

In [7]:
@view y[1:5]

5-element view(::Array{Float64,1}, 1:5) with eltype Float64:
 1.0
 2.0
 3.0
 4.0
 5.0

In [4]:
push!(v,@view y[6:10])

2-element Array{Any,1}:
 [1.0, 2.0, 3.0, 4.0, 5.0] 
 [6.0, 7.0, 8.0, 9.0, 10.0]

In [6]:
typeof(v[1])

SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true}

In [60]:
v[1][2] = 2.3

2.3

In [61]:
y

10-element Array{Float64,1}:
  1.0
  2.3
  3.0
  4.0
  5.0
  6.0
  7.7
  8.0
  9.0
 10.0